In [63]:
import requests

import re

from bs4 import BeautifulSoup

import pandas as pd

import json

from datetime import datetime

## Задание 1
Выполнить парсинг страницы со свежеми новостям на habr.com/ru/all/.

Необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [122]:
# список ключевых слов для поиска. Регистр неважен, лексический поиск НЕ выполняется - лучше записать корень слова
KEYWORDS = ['эпизод','Израиль','пандеми']

# подготовка регулярного выражения
regex = '\s{1}' + '|\s{1}'.join(KEYWORDS)
regex = regex.lower()
# print(regex)

habr_posts_with_keywords = pd.DataFrame()

# получение html целевой страницы
request = requests.get('https://habr.com/ru/all/')
request_html = BeautifulSoup(request.text, 'html.parser')
  
# поиск сегментов article, содержащих превью-информацию по каждому посту
posts = request_html.find_all('article', class_='post')
# len(posts)

# цикл, итерация - извлечение реквизитов превью и обработка
for post in posts:
    post_title = post.find('h2', class_='post__title').text
    post_time = post.find('span', class_='post__time').text
    post_link = post.find('a', class_='post__title_link').attrs.get('href')
    post_body = post.find('div', class_='post__text').text.lower()
    
#     поиск по регулярному выражению в тексте превью. Если поиск успешен - запись реквизитов в датафрейм
    search_keywords_res = re.findall(regex, post_body)
    if len(search_keywords_res) > 0:
#         print(post_time, post_title, post_link, search_keywords_res, post_body)
        
        # формирование строки датафрейма
        row = { 'time' : post_time, 'title' : post_title.strip(), 'link' : post_link }
        habr_posts_with_keywords = pd.concat([habr_posts_with_keywords, pd.DataFrame([row])])

habr_posts_with_keywords

,time,title,link
0,сегодня в 11:37,Северокорейская хакерская группа атаковала обо...,https://habr.com/ru/post/536062/
0,сегодня в 11:27,Космический 2021: Орбитальные станции и долгож...,https://habr.com/ru/post/536056/


## Задание 2

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [77]:
# список адресов для проверки с помощью сервиса
EMAIL = ['v.petrenko@personal-mode.com','jobs@apple.com']

# набор параметров для выполнения POST-запроса к сервису. Определен с помощью инструментов разработчика в браузере
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {
"Connection": "keep-alive",
"Content-Length": "32",
"Accept": "application/json, text/plain, */*",
"Vaar-Version": "0",
"Vaar-Header-App-Product": "hackcheck-web-avast",
"Content-Type": "application/json;charset=UTF-8",
"Origin": "https://www.avast.com",
"Sec-Fetch-Site": "same-site",
"Sec-Fetch-Mode": "cors",
"Sec-Fetch-Dest": "empty",
"Referer": "https://www.avast.com/hackcheck/",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "ru,en;q=0.9"
} 
payload = {}
payload['emailAddresses'] = EMAIL

# выполнение запроса и получение JSON.Интересующие сегменты - самый первый breaches и самый последний summary (смотреть соответственно первую и последнюю строки текста)
req = requests.post(URL, data=json.dumps(payload), headers=headers)
# req.text


In [78]:
# конвертация JSON в словарь
data_dict = json.loads(req.text)

# извлечение вложенных словарей с сегментами интересующих данных
data_dict_breaches = data_dict['breaches']
data_dict_summary = data_dict['summary']

emails = list(data_dict['summary'].keys())
email_breaches = pd.DataFrame()

# извлечение данных из словарей для каждого из адресов
for email in emails:
#     print(email)
    breaches = data_dict_summary[email]['breaches']
#     print(breaches)
    for breach in breaches:
#         print(breach)
        breach_dict = data_dict_breaches[str(breach)]
#         print(breach_dict)
        
        breach_source = breach_dict['site']
        breach_date = datetime.strftime(datetime.strptime(breach_dict['publishDate'], '%Y-%m-%dT%H:%M:%SZ'), '%Y-%m-%d')
        breach_description = breach_dict['description']
#         print(breach_source, breach_date, breach_description)
        
        # формирование строки датафрейма
        row = { 'email': email, 'date':breach_date, 'source':breach_source, 'description':breach_description }        
        email_breaches = pd.concat([email_breaches, pd.DataFrame([row])])
 
email_breaches

,email,date,source,description
0,v.petrenko@personal-mode.com,2019-03-20,bookmate.com,"In July 2018, Bookmate was allegedly breached...."
0,v.petrenko@personal-mode.com,2020-05-21,vk.com,"At some time in 2020, the Russian social netwo..."
0,v.petrenko@personal-mode.com,2016-10-21,myspace.com,"Shortly before the 2016 Memorial Day weekend, ..."
0,v.petrenko@personal-mode.com,2016-10-24,dropbox.com,Cloud storage company Dropbox suffered a major...
0,v.petrenko@personal-mode.com,2016-10-21,linkedin.com,"In 2012, online professional networking platfo..."
0,v.petrenko@personal-mode.com,2016-10-25,last.fm,"In March 2012, the British music streaming ser..."
0,v.petrenko@personal-mode.com,2019-12-13,peopledatalabs.com,"In October 2019, a security researcher discove..."
0,jobs@apple.com,2019-03-28,verifications.io,Big data e-mail verification platform verifica...
0,jobs@apple.com,2020-05-21,vk.com,"At some time in 2020, the Russian social netwo..."
0,jobs@apple.com,2018-02-18,disqus.com,"In 2012, comment hosting service Disqus's data..."
